# Analysis of the Runtime

In this notebook, we now analyze the performance of the different experiment-wares in terms of runtime.
More precisely, we only compare the experiment-wares based on the time they spent before completing their task (and thus, experiments in which the timeout is reached are considered unsuccessful, whatever their outcome).

## Imports

As usual, we start by importing the needed classes and functions from *Metrics-Wallet*.

In [ ]:
from metrics.wallet import BasicAnalysis, DecisionAnalysis, LineType
from metrics.wallet import find_best_cpu_time_input

## Loading the data of the experiments

In a [dedicated notebook]({{ load_notebook }}.ipynb), we already read and preprocessed the data collected during our experiments.
We can now simply reload the cached `BasicAnalysis` to retrieve it.

In [ ]:
basic_analysis = BasicAnalysis.import_from_file('.cache')

Since we now want to perform a more specific analysis, we need to create a `DecisionAnalysis` that will provide methods dedicated to the analysis of the runtime of experiments.

In [ ]:
analysis = DecisionAnalysis(basic_analysis=basic_analysis)

## Virtual Best Experiment-Ware

The *Virtual Best Experiment-Ware* (or VBEW) is an experiment-ware that does not really exist.
Its runtime on a particular input is that of the fastest experiment-ware that was run on that input (even though one could define a VBEW based on other criteria).
If one had an oracle to select the best experiment-ware for a particular input, and then run the experiment-ware on this input, its runtime would be that of the VBEW.

In [ ]:
analysis = analysis.add_virtual_experiment_ware(function=find_best_cpu_time_input, name='VBEW')

We can now compute the contribution of each experiment-ware to the VBEW.

In [ ]:
analysis.remove_experiment_wares(['VBEW'])\
        .contribution_table(deltas=(1, 10))

Let us describe how to read this table.
In the first column, we can see for each experiment-ware the number of inputs for which the runtime of the experiment-ware is equal to that of the VBEW.
In the second column (resp. third column), we can see for each experiment-ware the number of inputs for which the experiment-ware is at least 1 second faster (resp. 10 seconds faster) than any other experiment-ware.
Finally, in the fourth column, we can see the number of inputs for which this experiment-ware is the only one to run until completion (and thus, all other experiment-wares reached the time limit on this input).

## Overview of the results

An overview of the results can easily be obtained using a so-called *cactus-plot*, which is a figure that is particularly popular in the SAT or CP communities.

In [ ]:
analysis.cactus_plot(
    cactus_col='cpu_time',
    show_marker=False,

    title='Cactus-plot',
    x_axis_name='Number of solved inputs',
    y_axis_name='Time (s)',

    color_map={ 'VBS': '#000000' },
    style_map={ 'VBS': LineType.DASH_DOT },

    latex_writing=True
)

On this plot, we can easily read for each experiment-ware the number of inputs on which it can run until completion within a certain time limit.
In particular, the more an experiment-ware is to the right, the faster it is in general.

**TODO: ADD HERE AN INTERPRETATION FOR THIS CACTUS-PLOT!**


Another way to get an overview of the results is to use the *cumulative distribution function* (CDF), which may be seen as a cactus-plot in which the axes have been switched.

In [ ]:
analysis.cdf_plot(
    cdf_col='cpu_time',
    show_marker=False,
    normalized=True,

    title='CDF',
    x_axis_name='Time (s)',
    y_axis_name='Number of solved inputs',

    color_map={ 'VBS': '#000000' },
    style_map={ 'VBS': LineType.DASH_DOT },

    latex_writing=True
)

The interpretation of this plot is similar to that of a cactus-plot.
One of the advantage of this representation is that the order of the lines in the plot is the same as that of the legend, and thus best experiment-wares are on the top.
Additionally, it has more connections with the theory of statistics (while cactus-plots are not so meaningful outside the community).

Talking about statistics, box-plots can also be used to have an overview of the distribution of the runtime of the different experiment-wares.

In [ ]:
analysis.box_plot(
    box_by='experiment_ware',
    box_col='cpu_time',

    title='Box-plots of the runtime',

    latex_writing=True
)

**TODO: ADD HERE AN INTERPRETATION FOR THESE BOXPLOTS!**

## Pairwise comparisons

Now that we have an overview of the results, we can make a pairwise comparison of the experiment-wares, to have a closer look at their behavior.
We can do so by drawing so-called *scatter-plots*.

First, we need to select two of the experiment-wares among those run during the experiments.

In [ ]:
xp_ware_x = analysis.experiment_wares()[0]
xp_ware_y = analysis.experiment_wares()[1]

Once the experiment-wares have been selected, we can draw a scatter-plot that compares the runtime of both solvers on each input.
Here, each point is an input, and the x-axis and y-axis correspond to the runtime of `xp_ware_x` and `xp_ware_y` on this input, respectively.

In [ ]:
analysis.scatter_plot(
    xp_ware_x,
    xp_ware_y,

    scatter_col='cpu_time',
    title=f'Comparison between {xp_ware_x} and {xp_ware_y}',

    x_min=1,
    x_max={{ timeout }},
    y_min=1,
    y_max={{ timeout }},

    logx=True,
    logy=True,

    latex_writing=True
)

**TODO: ADD HERE AN INTERPRETATION FOR THIS SCATTER-PLOT!**

While the analysis presented in this template only presents one scatter-plot for demonstration purposes, it may be useful for you to draw more scatter-plots, based on your needs.
In fact, all pairwise comparisons between two experiment-wares could be visualized using scatter-plots.